# Model Building

### Importing libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Flatten, Dropout
from tensorflow.keras.layers import Convolution2D,MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!pip install keras==2.2.4

  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.


In [3]:
!pip install tensorflow

  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [4]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Vi1KVowrXT76o9nOsOvjlvhkOa9__CjF2BYq52pcQAnm',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-2koxwz4prbtxtk'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

### Augmenting the data 

In [6]:
train = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test = ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [7]:
x_train = train.flow_from_directory(r'Dataset/train',target_size=(64, 64),batch_size=3,color_mode='grayscale',class_mode='categorical')
x_test = test.flow_from_directory(r'Dataset/test',target_size=(64, 64),batch_size=3,color_mode='grayscale',class_mode='categorical') 

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [8]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


### Model Creation

In [9]:
model = Sequential()

**Adding Layers**

In [10]:
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
model.add(Convolution2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [12]:
model.add(Flatten())

In [13]:
model.add(Dense(units=512 , activation='relu'))

In [14]:
model.add(Dense(units=6, activation='softmax')) 

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

### Compile the Model

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fit the Model

In [18]:
model.fit(x_train,steps_per_epoch = 594/3 ,epochs = 25,validation_data = x_test,validation_steps = 30/3 )

Epoch 1/25
198/198 [==============================] - 13s 68ms/step - loss: 0.6684 - accuracy: 0.7239 - val_loss: 0.7362 - val_accuracy: 0.7000
Epoch 2/25
198/198 [==============================] - 13s 65ms/step - loss: 0.4460 - accuracy: 0.8300 - val_loss: 0.3181 - val_accuracy: 0.8667
Epoch 3/25
198/198 [==============================] - 13s 64ms/step - loss: 0.3373 - accuracy: 0.8754 - val_loss: 0.2893 - val_accuracy: 0.9667
Epoch 4/25
198/198 [==============================] - 13s 67ms/step - loss: 0.2133 - accuracy: 0.9293 - val_loss: 0.3781 - val_accuracy: 0.8000
Epoch 5/25
198/198 [==============================] - 13s 65ms/step - loss: 0.1861 - accuracy: 0.9259 - val_loss: 0.1605 - val_accuracy: 0.9667
Epoch 6/25
198/198 [==============================] - 13s 65ms/step - loss: 0.1471 - accuracy: 0.9461 - val_loss: 0.1437 - val_accuracy: 0.9667
Epoch 7/25
198/198 [==============================] - 13s 65ms/step - loss: 0.1412 - accuracy: 0.9444 - val_loss: 0.1625 - val_accuracy:

### Save the Model

In [19]:
model.save('gesture.h5')

In [20]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# Save The Model in tar file format

In [45]:
!tar -zcvf Gesture.tgz gesture.h5

gesture.h5


# IBM Deployment

In [23]:
!pip install ibm_watson_machine_learning

In [37]:
from ibm_watson_machine_learning import APIClient

API_KEY = "F2CZbqXAjry_1Q21z6pxFMcfRqO6XM-ulLPWfD8lkMFi"

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(wml_credentials)

In [38]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [40]:
space_uid="d9ec3745-2e00-46d1-9c4a-c6ab5e012501"
print("Space UID "+space_uid)

Space UID d9ec3745-2e00-46d1-9c4a-c6ab5e012501


In [41]:
client.set.default_space(space_uid)

'SUCCESS'

In [42]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [43]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [50]:
model_details = client.repository.store_model(model="Gesture.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "Models",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [51]:
model_id

'cb31e9e2-b791-4e9d-9874-e555caaff96c'

In [53]:
client.repository.download(model_id, "model.tar.gz")

File with name: 'model.tar.gz' already exists.


WMLClientError: File with name: 'model.tar.gz' already exists.